# Исследование надёжности заёмщиков

Исследовать, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Результаты анализа в дальнейшем будут использованы кредитным отделом банка при построении модели кредитного скоринга.

### Описание признаков:

- children — количество детей в семье;
- days_employed — общий трудовой стаж в днях;
- dob_years — возраст клиента в годах;
- education — уровень образования клиента;
- education_id — идентификатор уровня образования;
- family_status — семейное положение;
- family_status_id — идентификатор семейного положения;
- gender — пол клиента;
- income_type — тип занятости;
- debt — имел ли задолженность по возврату кредитов;
- total_income — ежемесячный доход;
- purpose — цель получения кредита.

### Библиотеки и модули

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

### Содержание

1. Изучение общей информации о данных
2. Предобработка данных:
    - Обработка пропусков
    - Замена типа данных
    - Обработка дубликатов
    - Лемматизация
    - Категоризация данных
    - Дополнительное изучение данных
3. Аналитика:
    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?
4. Общий вывод

## Изучение общей информации о данных

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Вывод

В общей информации данной таблицы видно, что есть столбцы с пропусками (days_employed, total_income), так как 19351 значений в них заполнено в то время, как в остальных столбцах - 21525. Также интересно то, что в столбце days_employed содержаться отрицательные значения...

## Предобработка данных

### Обработка пропусков

Пропуски могут появится из-за технических причин или человеческого фактора.

Пропуски разделяют на три вида:
- полностью случайные - пропуски, вероятность появления которых не зависит от других значений в наборе данных
- случайные - пропуски, вероятность появления которых зависит от других значений в наборе данных
- неслучайные - пропуски, вероятность появление которых зависит от других значений и от значения собственного столбца

In [4]:
data.isnull().sum() 
# комбинация методов isnull и sum дает информацию о том, сколько пропусков в каждом столбце

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Первый вопрос, на который нужно ответить, есть ли закономерность в появлении пропусков в столбцах days_employed и total_income?

- видно, что пропуски допущены в количественных переменных, так тип данных этих столбцов float64
- предполагаю, что между пропусками в этих столбцах существует закономерность

Гипотеза: если человек не указывает то, сколько дней он проработал, значит он не работал, поэтому он не заполняет поле дохода. Исходя из этих рассуждений, пропуски в столбце days_employed полностью случайные, а в столбце total_income - случайные.

In [5]:
data.loc[data['days_employed'].isnull() == True].head(10) 
# вывод строк в таблице, в которых содержаться пропуски в столбце days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Анализируя данные есть прямая закономерность между тем, что не заполнено поле days_employed и total_income. Однако, предположение о том, что человек не работал ни дня, поэтому не указал доход не верна, потому что, если посмотрев на источники дохода людей в этой выборке, то там будут: сотрудник, пенсионер, госслужащий и др.. В таком случае человек не мог не работать и не получать доход. Тогда можно рассуждать от идеи, что люди скрывают свой доход и не желают его говорить, как и количество проработанных дней, или один из работников, работающих в банке, собирая эту информацию о клиентах, забыл занести ее в таблицу, или просто не спросил клиента об этом. Не исключено технической причины.

Заполню пропуски в столбце total_income медианной велечиной для каждой из категорий источников дохода, предварительно сгруппировав по income_type, так как на одном и том же источнике дохода может быть сильный разброс в значениях, пропуски в столбце days_employed - также медианной велечиной.

Однако вспоминаем, что в столбце days_employed содержаться отрицательные значения, чего не может быть по смыслу переменной. Возможно данная ошибка была допущена из-за технических проблем, что при выгрузке добавлялся минус к полям этого столбца. Изменю отрицательные значения на положительные.

In [6]:
def abs_days_employed(var): # функиця, которая выдает значения модуля переданного числа
    return abs(var)

data['days_employed'] = data.loc[data['days_employed'].isnull() == False, 'days_employed'].apply(abs_days_employed)
# применяю функцию abs_days_employed с помощью метода apply к столбцу days_employed

In [7]:
dict_income_type = data.groupby('income_type').agg({'total_income': ['median'], 'days_employed': ['median'] })
# группирую таблицу по столбцу income_type с помощью метода groupby и агрегирую с помощью метода agg
dict_income_type

,total_income,days_employed
,median,median
income_type,,
безработный,131339.751676,366413.652744
в декрете,53829.130729,3296.759962
госслужащий,150447.935283,2689.368353
компаньон,172357.950966,1547.382223
пенсионер,118514.486412,365213.306266
предприниматель,499163.144947,520.848083
сотрудник,142594.396847,1574.202821
студент,98201.625314,578.751554


In [8]:
data = data.fillna(value='') # заменяю пропуски на ''
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.186,27,высшее,0,гражданский брак,1,M,компаньон,0,255764,покупка жилья
6,0,2879.2,43,высшее,0,женат / замужем,0,F,компаньон,0,240526,операции с жильем
7,0,152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135824,образование
8,2,6929.87,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.8,на проведение свадьбы
9,0,2188.76,41,среднее,1,женат / замужем,0,M,сотрудник,0,144426,покупка жилья для семьи


In [9]:
data_len = data.shape[0]

for x in range(data_len): # произвожу замену пустых значений на медианные значения ежемесячного дохода
    try:
        if data.loc[x, 'total_income'] == '':
            x_income_type = data.loc[x, 'income_type']
            median_income = dict_income_type.loc[x_income_type, 'total_income'].to_list()[0]
            data.loc[x, 'total_income'] = median_income
    except:
        print('Error')

Использую конструкцию try-except для того, чтобы нивелировать появление ошибок. Данный алгоритм делают проверку для каждой строки на то, есть ли в столбце 'total_income' пропущенное значение. Если значение пропущено, то сохраняем в переменную x_income_type тип источника дохода. После обращаюсь к словарю источник тип занятости-медианное значение ежемесячного дохода для данного типа занятости и выбираем медианный ежемесячный доход (median_income_type), который вставляю в значение данной строки в столбец 'total_income'.

In [10]:
data.loc[data['total_income'] == ''] # проверяю на наличие пустых значений в столбце total_income

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [11]:
data.loc[data['days_employed'] == '', ['days_employed', 'total_income']].head(10)

,days_employed,total_income
12,,118514
26,,150448
29,,118514
41,,150448
55,,118514
65,,172358
67,,118514
72,,150448
82,,142594
83,,142594


In [12]:
data_len = data.shape[0]

for x in range(data_len): # произвожу замену пропусков на медианное значение стажа работы
    try:
        if data.loc[x, 'days_employed'] == '':
            x_income_type = data.loc[x, 'income_type']
            median_days_employed = dict_income_type.loc[x_income_type, 'days_employed'].to_list()[0]
            data.loc[x, 'days_employed'] = median_days_employed
    except:
        print('Error')

In [13]:
data.loc[data['days_employed'] == '', ['days_employed', 'total_income']].head(10)
# проверяю есть ли пустые значения в столбце days_employed

,days_employed,total_income


#### Вывод

Итого, произвел замену пропусков в столбцах total_income и days_employed, заполняя их медианным значениям, сгруппировав по источнику дохода (income_type). Установил, что закономерность между пропусками в этих столбцах закономерна. Также были подкорректрированы данные в столбце days_employed с отрицательных значений на положительные.

### Замена типа данных

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null object
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null object
purpose             21525 non-null object
dtypes: int64(5), object(7)
memory usage: 2.0+ MB


Видно, что столбец days_employed и total_income нужно перевести из типа строкового в тип float64, так как в них содержаться данные числового типа.

In [15]:
try:
    data['days_employed'] = pd.to_numeric(data['days_employed']) 
    data['total_income'] = pd.to_numeric(data['total_income'])
except:
    print('Error')

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Вывод

Заменены типы данных со строчных на float64 столбцов: days_employed и total_income.

### Обработка дубликатов

Искать дубликаты стоит в столбцах: education, family_status, gender, income_type. Остальные типы столбцов обладют количественным типом данных, столбец purpose содержит текстовую причину.

In [17]:
# рассматриваю столбец education
print(data['education'].duplicated().sum())
print(data['education'].value_counts())

21510
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


Столбец содержит значения одного и тоже слова, написанного символами разного регистра. Переведем все символы значений в этом столбце в нижний регистр.

In [18]:
data['education'] = data['education'].str.lower() 
# использую метод str.lower для того, чтобы привести все значения в столбце к нижнему регистру
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [19]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

По сути гражданский брак и не женат / не замужем это одно и тоже, поэтому оставим только не женат / не замужем.

In [20]:
data.loc[data['family_status'] == 'гражданский брак', 'family_status'] = 'женат / замужем'
data['family_status'].value_counts()

женат / замужем          16557
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [21]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Интересно, что есть одна строка, в которой содержится пол, указанный как XNA. Что это? Заменим его на значение моды.

In [22]:
data.loc[data['gender'] == 'XNA', 'gender'] = data['gender'].mode()
data['gender'] = data['gender'].str.lower()
data.loc[data['gender'] == 'XNA', 'gender']

Series([], Name: gender, dtype: object)

In [23]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [24]:
data.duplicated().sum()

71

In [25]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

#### Вывод

Были убраны лишние значения в столбце education. Убрано одно значение в столбце gender. Поскольку значения в столбцах категориальные, то дубликатов в общем смысле искать здесь не нужно.

### Лемматизация

Лемматизация - процесс приведения слова к слова к словарной форме.

In [26]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
words = []
rows = data.shape[0]
try: # создаю список уникальных словарных слов из всех причин взятия кредит
    for row in range(rows):
        list_purpose = data.loc[row, 'purpose']
        words += m.lemmatize(data.loc[row, 'purpose'])
except:
    print('Error')

words = Counter(words)
print(words.keys())

dict_keys(['покупка', ' ', 'жилье', '\n', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'с', 'на', 'проведение', 'для', 'семья', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'собственный', 'подержать', 'свой', 'со', 'заниматься', 'сделка', 'подержанный', 'получение', 'высокий', 'профильный', 'сдача', 'ремонт'])


In [27]:
list_target_words = ['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость', 'строительство', 'ремонт']
# из полученных уникальных создаю список, в котором находятся важные и ключевые слова, которые характеризуют причину

In [28]:
rows = data.shape[0]

for row in range(rows):
    try: # создаю столбец, в котором будет храниться ключевое слово, характеризующее причину
        lemmas = m.lemmatize(data.loc[row, 'purpose'])
        for lemma in lemmas:
            if lemma in list_target_words:
                data.loc[row, 'purpose_unique'] = lemma
    except:
        print('Error')

In [29]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,жилье
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,жилье
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,женат / замужем,1,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,926.185831,27,высшее,0,женат / замужем,1,m,компаньон,0,255763.565419,покупка жилья,жилье
6,0,2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем,жилье
7,0,152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование,образование
8,2,6929.865299,35,высшее,0,женат / замужем,1,f,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи,жилье


In [31]:
data.loc[data['purpose_unique'] == 'жилье', 'purpose_unique'] = 'недвижимость'
print(data['purpose_unique'].value_counts())
print(f"Сумма значений равна: {data['purpose_unique'].value_counts().sum()}")
print(f'Всего строк в таблице: {data.shape[0]}')

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_unique, dtype: int64
Сумма значений равна: 21454
Всего строк в таблице: 21454


#### Вывод

Имеем столбец, в котором одним словом указана причина взятия кредита из предложения клиента.

### Категоризация данных

Категоризация данных делится на два типа: категоризация по типу и категоризация при объединении данных.

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
children            21454 non-null int64
days_employed       21454 non-null float64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21453 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
purpose_unique      21454 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Под категоризацию по типу попадают следующие столбцы таблицы: family_status, education, income_type, purpose_unique.
Под категоризацию по объединению данных попадают следующие столбцы: children, dob_years.

В столбце purpose_unique назовем одним словом: недвижимость и жилье.

In [33]:
data.loc[data['purpose_unique'] == 'жилье', 'purpose_unique'] = 'недвижимость'
data['purpose_unique'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_unique, dtype: int64

Проделаем следующую категоризацию данных - по количеству детей в семье можно разделить ее на бездетную (0 детей), малодетную (1-2 ребенка) и многодетную (3+ детей).

Есть ли у людей не женатых / не замужем дети?

In [34]:
data.loc[data['family_status'] == 'не женат / не замужем', 'children'].value_counts()

 0     2262
 1      449
 2       75
 20       9
 3        8
-1        5
 4        2
Name: children, dtype: int64

Да, дети есть, а у кого-то даже 20 детей! у кого-то -1 ребенок :) Стоит посмотреть на всех людей, а не на выборку не женатых / не замужем людей.

In [35]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

20 детей можно постараться объяснить очень просто: при вводе данных или выгрузке данных к числу 2 был добавлен 0. Где же -1 ребенок был приписан '-' при выгрузке-загрузке данных. Как та, так и другая ошибки могли быть результатом технической проблемы или человеческого фактора. Исправим это.

In [36]:
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [37]:
# Считаю, что полная семья: есть один и более ребенок и семейны статус: женат / не женат, гражданский брак, не женат / не замужем.
# Неполная семья: есть один и более ребенок и семейный статус: в разводе, вдовец / вдова. Нет семьи: нет ребенка и семейный статус: любой.
# Бездтеная семья: 0 детей, малодетная семья: 1-2 ребенка, многодетная семья: 3+ ребенка.
def family_type_by_children_parents(row):
    number_of_childrens = row['children']
    family_status = row['family_status']
    if number_of_childrens == 0:
        return 'Не семья'
    if 1 <= number_of_childrens <= 2:
        if family_status in ['женат / замужем', 'не женат / не замужем']:
            return 'полная малодетная семья'
        return 'неполная малодетная семья'
    if number_of_childrens > 2:
        if family_status in ['женат / замужем', 'не женат / не замужем']:
            return 'полная многодетная семья'
        return 'неполная многодетная семья'
    
data['family_type_by_children_parents'] = data.apply(family_type_by_children_parents, axis=1)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,family_type_by_children_parents
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,недвижимость,Не семья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,образование,полная многодетная семья
4,0,340266.072047,53,среднее,1,женат / замужем,1,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,Не семья
5,0,926.185831,27,высшее,0,женат / замужем,1,m,компаньон,0,255763.565419,покупка жилья,недвижимость,Не семья
6,0,2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем,недвижимость,Не семья
7,0,152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование,образование,Не семья
8,2,6929.865299,35,высшее,0,женат / замужем,1,f,сотрудник,0,95856.832424,на проведение свадьбы,свадьба,полная малодетная семья
9,0,2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость,Не семья


In [38]:
data['family_type_by_children_parents'].value_counts()

Не семья                      14091
полная малодетная семья        6479
неполная малодетная семья       504
полная многодетная семья        361
неполная многодетная семья       19
Name: family_type_by_children_parents, dtype: int64

Избавимся от избыточности и упростим работу с данными. Начнем с family_status.

In [39]:
dict_family_status = data[['family_status', 'family_status_id']] # словарь, в котором хранится статус семьи и ее индекс
dict_family_status.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,женат / замужем,1
5,женат / замужем,1
6,женат / замужем,0
7,женат / замужем,0
8,женат / замужем,1
9,женат / замужем,0


In [40]:
dict_family_status = dict_family_status.drop_duplicates().reset_index(drop=True)
dict_family_status.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Странно, что женат / замужем повторился два раза. Это ошибка возникла из-за того, что гражданский брак был заменен, а его id - нет.

In [41]:
data.loc[data['family_status_id'] == 1, 'family_status_id'] = 0
dict_family_status = data[['family_status', 'family_status_id']]
dict_family_status = dict_family_status.drop_duplicates().reset_index(drop=True)
dict_family_status.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,вдовец / вдова,2
2,в разводе,3
3,не женат / не замужем,4


In [42]:
data = data.drop(['family_status'], axis='columns')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,family_type_by_children_parents
0,1,8437.673028,42,высшее,0,0,f,сотрудник,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья
1,1,4024.803754,36,среднее,1,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья
2,0,5623.422610,33,среднее,1,0,m,сотрудник,0,145885.952297,покупка жилья,недвижимость,Не семья
3,3,4124.747207,32,среднее,1,0,m,сотрудник,0,267628.550329,дополнительное образование,образование,полная многодетная семья
4,0,340266.072047,53,среднее,1,0,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,Не семья


Аналогично с education.

In [43]:
try:
    dict_education = data[['education', 'education_id']]
    dict_education = dict_education.drop_duplicates().reset_index(drop=True)
    data = data.drop(['education'], axis='columns')
except:
    print('That operations already have been done.')

In [44]:
dict_education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [45]:
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,family_type_by_children_parents
0,1,8437.673028,42,0,0,f,сотрудник,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья
1,1,4024.803754,36,1,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья
2,0,5623.422610,33,1,0,m,сотрудник,0,145885.952297,покупка жилья,недвижимость,Не семья
3,3,4124.747207,32,1,0,m,сотрудник,0,267628.550329,дополнительное образование,образование,полная многодетная семья
4,0,340266.072047,53,1,0,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,Не семья


Аналогичные операции проделаю со столбцами income_type, purpose_unique и family_type_by_children_parents, только перед этим добавив столбцы с id в таблицу data.

In [46]:
list_income_type = data['income_type'].unique()
for i in range(len(list_income_type)):
    data.loc[data['income_type'] == list_income_type[i], 'income_type_id'] = i
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_unique,family_type_by_children_parents,income_type_id
0,1,8437.673028,42,0,0,f,сотрудник,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья,0.0
1,1,4024.803754,36,1,0,f,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья,0.0
2,0,5623.422610,33,1,0,m,сотрудник,0,145885.952297,покупка жилья,недвижимость,Не семья,0.0
3,3,4124.747207,32,1,0,m,сотрудник,0,267628.550329,дополнительное образование,образование,полная многодетная семья,0.0
4,0,340266.072047,53,1,0,f,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,Не семья,1.0


In [47]:
data['income_type_id'] = data['income_type_id'].astype('int') # делаю замену типа данных с вещественного на целочисленный
dict_income_type = data[['income_type', 'income_type_id']]
dict_income_type = dict_income_type.drop_duplicates().reset_index(drop=True)
data = data.drop(['income_type'], axis='columns')
dict_income_type

,income_type,income_type_id
0,сотрудник,0
1,пенсионер,1
2,компаньон,2
3,госслужащий,3
4,безработный,4
5,предприниматель,5
6,студент,6
7,в декрете,7


In [48]:
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,purpose,purpose_unique,family_type_by_children_parents,income_type_id
0,1,8437.673028,42,0,0,f,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья,0
1,1,4024.803754,36,1,0,f,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья,0
2,0,5623.422610,33,1,0,m,0,145885.952297,покупка жилья,недвижимость,Не семья,0
3,3,4124.747207,32,1,0,m,0,267628.550329,дополнительное образование,образование,полная многодетная семья,0
4,0,340266.072047,53,1,0,f,0,158616.077870,сыграть свадьбу,свадьба,Не семья,1


purpose_unique

Напишу функции для упрощения работы с другими столбцами.

In [49]:
def id_column_for_categorical_column(name, df):
    list_column_name = df[name].unique()
    for i in range(len(list_column_name)):
        df.loc[df[name] == list_column_name[i], f'{name}_id'] = i
    df[f'{name}_id'] = df[f'{name}_id'].astype('int')
    return df
    
data = id_column_for_categorical_column('purpose_unique', data)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,purpose,purpose_unique,family_type_by_children_parents,income_type_id,purpose_unique_id
0,1,8437.673028,42,0,0,f,0,253875.639453,покупка жилья,недвижимость,полная малодетная семья,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,приобретение автомобиля,автомобиль,полная малодетная семья,0,1
2,0,5623.422610,33,1,0,m,0,145885.952297,покупка жилья,недвижимость,Не семья,0,0
3,3,4124.747207,32,1,0,m,0,267628.550329,дополнительное образование,образование,полная многодетная семья,0,2
4,0,340266.072047,53,1,0,f,0,158616.077870,сыграть свадьбу,свадьба,Не семья,1,3


In [50]:
def create_dict_for_categorical_column(id_column, column, df):
    dict_column = df[[column, id_column]]
    dict_column = dict_column.drop_duplicates().reset_index(drop=True)
    return dict_column

dict_purpose_unique = create_dict_for_categorical_column('purpose_unique', 'purpose_unique_id', data)
dict_purpose_unique

,purpose_unique_id,purpose_unique
0,0,недвижимость
1,1,автомобиль
2,2,образование
3,3,свадьба


In [51]:
data = data.drop(['purpose', 'purpose_unique'], axis='columns')
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,family_type_by_children_parents,income_type_id,purpose_unique_id
0,1,8437.673028,42,0,0,f,0,253875.639453,полная малодетная семья,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,полная малодетная семья,0,1
2,0,5623.422610,33,1,0,m,0,145885.952297,Не семья,0,0
3,3,4124.747207,32,1,0,m,0,267628.550329,полная многодетная семья,0,2
4,0,340266.072047,53,1,0,f,0,158616.077870,Не семья,1,3


family_type_by_children_parents

In [52]:
data = id_column_for_categorical_column('family_type_by_children_parents', data)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,family_type_by_children_parents,income_type_id,purpose_unique_id,family_type_by_children_parents_id
0,1,8437.673028,42,0,0,f,0,253875.639453,полная малодетная семья,0,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,полная малодетная семья,0,1,0
2,0,5623.422610,33,1,0,m,0,145885.952297,Не семья,0,0,1
3,3,4124.747207,32,1,0,m,0,267628.550329,полная многодетная семья,0,2,2
4,0,340266.072047,53,1,0,f,0,158616.077870,Не семья,1,3,1


In [53]:
dict_family_type_by_children_parents = create_dict_for_categorical_column('family_type_by_children_parents', 'family_type_by_children_parents_id', data)
dict_family_type_by_children_parents

,family_type_by_children_parents_id,family_type_by_children_parents
0,0,полная малодетная семья
1,1,Не семья
2,2,полная многодетная семья
3,3,неполная малодетная семья
4,4,неполная многодетная семья


In [54]:
data = data.drop(['family_type_by_children_parents'], axis='columns')
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id
0,1,8437.673028,42,0,0,f,0,253875.639453,0,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,0,1,0
2,0,5623.422610,33,1,0,m,0,145885.952297,0,0,1
3,3,4124.747207,32,1,0,m,0,267628.550329,0,2,2
4,0,340266.072047,53,1,0,f,0,158616.077870,1,3,1


#### Вывод

Сделана категоризация семей по количеству детей и родителей и категоризация по причине взятия кредита. Также были вынесены в отдельные таблицы категории по типам: education, family_status, income_type, purpose_unique, family_type_by_children_parents.

### Дополнительное изучение данных

Осталось поближе познакомиться с данными в столбцах days_employed и dob_years. Начну с days_employed.

Очевидно, что у человека не может быть общий трудовой стаж в днях по крайней мере больше 30000 дней точно (это 83+ года работы каждый день). Снизим пограничное значение до 10000 дней (27+ лет работы каждый день).

In [55]:
data.loc[data['days_employed'] >= 10000].shape

(4091, 11)

Посмотрим на значения, у которых >= 100000.

In [56]:
data.loc[data['days_employed'] >= 100000].shape

(3831, 11)

Здесь явно допущена ошибка при выгрузке-загрузке данных. Потому что такой стажа работы не может быть. Чтобы исправить эту ошибку поделим каждое числое в этой выборке на 100.

In [57]:
data.loc[data['days_employed'] >= 100000, 'days_employed'] = data.loc[data['days_employed'] >= 100000, 'days_employed'] / 100
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id
0,1,8437.673028,42,0,0,f,0,253875.639453,0,0,0
1,1,4024.803754,36,1,0,f,0,112080.014102,0,1,0
2,0,5623.422610,33,1,0,m,0,145885.952297,0,0,1
3,3,4124.747207,32,1,0,m,0,267628.550329,0,2,2
4,0,3402.660720,53,1,0,f,0,158616.077870,1,3,1


In [58]:
data.loc[data['days_employed'] >= 15000]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id
1539,0,15785.678893,59,0,4,f,0,119563.851852,0,0,1
3972,0,15835.725775,64,1,0,f,0,96858.531436,2,3,1
4296,0,17615.563266,61,1,0,f,0,122560.741753,2,0,1
4318,0,15773.061335,61,1,0,f,0,205868.585780,0,3,1
5576,0,15079.216069,55,1,0,f,0,178761.373413,3,0,1
7323,0,16593.472817,60,0,0,f,0,124697.846781,0,2,1
7725,0,15618.063786,64,1,0,f,0,296525.358574,2,2,1
15640,0,15410.040779,65,0,0,f,0,188800.068859,0,0,1
16297,1,18388.949901,61,1,0,f,0,186178.934089,0,0,0
16786,0,16119.687737,64,1,0,f,0,91527.685995,0,0,1


Допустим, что такое может быть. Критичным было бы значение 20000, но таких данных нет, к счастью :)

In [59]:
data.loc[data['days_employed'] == 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


Теперь изучим столбец dob_years.

In [60]:
data.loc[data['dob_years'] >= 100]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


In [61]:
data.loc[data['dob_years'] < 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


In [62]:
data.loc[data['dob_years'] == 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id
99,0,3465.416189,0,1,0,f,0,71291.522491,1,1,1
149,0,2664.273168,0,1,3,f,0,70176.435951,0,0,1
270,3,1872.663186,0,1,0,f,0,102166.458894,0,0,2
578,0,3978.565650,0,1,0,f,0,97620.687042,1,0,1
1040,0,1158.029561,0,0,3,f,0,303994.134987,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
19770,0,1574.202821,0,1,0,f,0,142594.396847,0,0,1
20397,0,3387.348685,0,1,0,f,0,259193.920299,1,0,1
20512,0,3317.412715,0,1,4,f,0,129788.762899,1,0,1
21110,2,108.967042,0,0,0,m,0,240702.007382,2,0,0


Заменим на среднее значение возраста там, где возраст равен 0.

In [63]:
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].mean()
data.loc[data['dob_years'] == 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


In [64]:
data.loc[data['total_income'] < 0]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id


## Аналитика

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [65]:
data.groupby('debt')['debt'].count()

debt
0    19713
1     1741
Name: debt, dtype: int64

In [66]:
def is_children(row):
    if row == 0:
        return 'Нет детей'
    return 'Есть дети'

data['is_children'] = data['children'].apply(is_children)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id,is_children
0,1,8437.673028,42.0,0,0,f,0,253875.639453,0,0,0,Есть дети
1,1,4024.803754,36.0,1,0,f,0,112080.014102,0,1,0,Есть дети
2,0,5623.422610,33.0,1,0,m,0,145885.952297,0,0,1,Нет детей
3,3,4124.747207,32.0,1,0,m,0,267628.550329,0,2,2,Есть дети
4,0,3402.660720,53.0,1,0,f,0,158616.077870,1,3,1,Нет детей


In [67]:
def is_debt(row):
    if row == 1:
        return 'Есть задолженность'
    return 'Нет задолженности'

data['is_debt'] = data['debt'].apply(is_debt)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id,is_children,is_debt
0,1,8437.673028,42.0,0,0,f,0,253875.639453,0,0,0,Есть дети,Нет задолженности
1,1,4024.803754,36.0,1,0,f,0,112080.014102,0,1,0,Есть дети,Нет задолженности
2,0,5623.422610,33.0,1,0,m,0,145885.952297,0,0,1,Нет детей,Нет задолженности
3,3,4124.747207,32.0,1,0,m,0,267628.550329,0,2,2,Есть дети,Нет задолженности
4,0,3402.660720,53.0,1,0,f,0,158616.077870,1,3,1,Нет детей,Нет задолженности


In [68]:
data_pivot_children = data.pivot_table(index='is_children', columns='is_debt', values='debt', aggfunc='count')
data_pivot_children

is_debt,Есть задолженность,Нет задолженности
is_children,,
Есть дети,678,6685
Нет детей,1063,13028


Найдем долю в каждой из категорий (Есть дети, Нет детей) долю кредитов с задолженностью в сумме общего количества взятых кредитов в данной категории.

In [69]:
data_pivot_children['ratio_expired'] = data_pivot_children['Есть задолженность'] / (data_pivot_children['Нет задолженности'] + data_pivot_children['Есть задолженность'])
data_pivot_children

is_debt,Есть задолженность,Нет задолженности,ratio_expired
is_children,,,
Есть дети,678,6685,0.092082
Нет детей,1063,13028,0.075438


Из сводной таблицы следует вывод, что доля задолженности у людей, у которых есть ребенок, выше, чем у кого его нет, на 0.016791 (или 1,6791%).

А посмотрим на доли для категорий family_type_by_children_parents_id.

In [70]:
dict_family_type_by_children_parents

,family_type_by_children_parents_id,family_type_by_children_parents
0,0,полная малодетная семья
1,1,Не семья
2,2,полная многодетная семья
3,3,неполная малодетная семья
4,4,неполная многодетная семья


In [71]:
data_pivot_children_2 = data.pivot_table(index='family_type_by_children_parents_id', columns='is_debt', values='debt', aggfunc='count')
data_pivot_children_2['ratio_expired'] = data_pivot_children_2['Есть задолженность'] / (data_pivot_children_2['Есть задолженность'] + data_pivot_children_2['Нет задолженности'])
data_pivot_children_2.sort_values(by='ratio_expired', ascending=False)

is_debt,Есть задолженность,Нет задолженности,ratio_expired
family_type_by_children_parents_id,,,
0,608,5871,0.093842
2,30,331,0.083102
3,39,465,0.077381
1,1063,13028,0.075438
4,1,18,0.052632


Из этой сводной таблицы видно, что наибольшая доля задолженности у полных малодетных семей (9.3654%). На втором месте - полные многодетные семьи (8.3102%). На третьем - неполные малодетные семьи (7.7381%). На четвертом - не семья (7.5129%). И на пятом - неполные многодетные семьи (5.2632%).

#### Вывод

Делаем вывод, что у кого есть дети, больше склоны к задолжностям по кредиту. Полные малодетных и многодетные семьи больше всего склоны к тому, чтобы иметь задолженность по кредиту. После неполные малодетные семьи и не семьи имеют одинаковую долю задолженности. И наиименее склоны к задолженности неполные многодетные сеьми. Интересно-интересно :)

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [72]:
data_pivot_family_status = data.pivot_table(index='family_status_id', columns='is_debt', values='debt', aggfunc='count')
data_pivot_family_status['ratio_expired'] = data_pivot_family_status['Есть задолженность'] / (data_pivot_family_status['Есть задолженность'] + data_pivot_family_status['Нет задолженности'])
data_pivot_family_status.sort_values(by='ratio_expired', ascending=False)

is_debt,Есть задолженность,Нет задолженности,ratio_expired
family_status_id,,,
4,274,2536,0.097509
0,1319,15171,0.079988
3,85,1110,0.071130
2,63,896,0.065693


In [73]:
dict_family_status

,family_status,family_status_id
0,женат / замужем,0
1,вдовец / вдова,2
2,в разводе,3
3,не женат / не замужем,4


#### Вывод

Из сводной таблицы можно сделать вывод, что к задолженности больше склоны не женатые / не замужем люди (9.7405%). Люди, которые либо поженились или вступили в гражданской брак, имею долю задолженности 7.9664%. Почти такой процент у тех людей, кто в разводе (7.113%). Меньше всего к задолженности склоны люди, являющиеся вдовцами / вдовами - 6.5625%.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Чтобы это выяснить, нужно прокатегоризировать людей по уровню дохода.

In [74]:
def level_income(column_value):
    if column_value <= 100000:
        return 'низкий доход'
    if 100000 < column_value <= 300000:
        return 'средний доход'
    return 'высокий доход'

data['level_income'] = data['total_income'].apply(level_income)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_unique_id,family_type_by_children_parents_id,is_children,is_debt,level_income
0,1,8437.673028,42.0,0,0,f,0,253875.639453,0,0,0,Есть дети,Нет задолженности,средний доход
1,1,4024.803754,36.0,1,0,f,0,112080.014102,0,1,0,Есть дети,Нет задолженности,средний доход
2,0,5623.422610,33.0,1,0,m,0,145885.952297,0,0,1,Нет детей,Нет задолженности,средний доход
3,3,4124.747207,32.0,1,0,m,0,267628.550329,0,2,2,Есть дети,Нет задолженности,средний доход
4,0,3402.660720,53.0,1,0,f,0,158616.077870,1,3,1,Нет детей,Нет задолженности,средний доход
5,0,926.185831,27.0,0,0,m,0,255763.565419,2,0,1,Нет детей,Нет задолженности,средний доход
6,0,2879.202052,43.0,0,0,f,0,240525.971920,2,0,1,Нет детей,Нет задолженности,средний доход
7,0,152.779569,50.0,1,0,m,0,135823.934197,0,2,1,Нет детей,Нет задолженности,средний доход
8,2,6929.865299,35.0,0,0,f,0,95856.832424,0,3,0,Есть дети,Нет задолженности,низкий доход
9,0,2188.756445,41.0,1,0,m,0,144425.938277,0,0,1,Нет детей,Нет задолженности,средний доход


In [75]:
data['level_income'].value_counts()

средний доход    15508
низкий доход      4463
высокий доход     1483
Name: level_income, dtype: int64

In [76]:
data_pivot_level_income = data.pivot_table(index='level_income', columns='is_debt', values='debt', aggfunc='count')
data_pivot_level_income['ratio_expired'] = data_pivot_level_income['Есть задолженность'] / (data_pivot_level_income['Есть задолженность'] + data_pivot_level_income['Нет задолженности'])
data_pivot_level_income.sort_values(by='ratio_expired', ascending=False)

is_debt,Есть задолженность,Нет задолженности,ratio_expired
level_income,,,
средний доход,1281,14227,0.082603
низкий доход,354,4109,0.079319
высокий доход,106,1377,0.071477


#### Вывод

Видим, что у людей с высоким доходом самая низкая доля задолженности - 7.1477%. Больше всего склонен к задолженности средний класс - 8.2226%. Процент людей с низким доходом, которые имеют задолженность по кредиту, равен 7.9319%.

### Как разные цели кредита влияют на его возврат в срок?

In [77]:
data_pivot_level_income = data.pivot_table(index='purpose_unique_id', columns='is_debt', values='debt', aggfunc='count')
data_pivot_level_income['ratio_expired'] = data_pivot_level_income['Есть задолженность'] / (data_pivot_level_income['Есть задолженность'] + data_pivot_level_income['Нет задолженности'])
data_pivot_level_income.sort_values(by='ratio_expired', ascending=False)

is_debt,Есть задолженность,Нет задолженности,ratio_expired
purpose_unique_id,,,
1,403,3903,0.093590
2,370,3643,0.092200
3,186,2138,0.080034
0,782,10029,0.072334


In [78]:
dict_purpose_unique

,purpose_unique_id,purpose_unique
0,0,недвижимость
1,1,автомобиль
2,2,образование
3,3,свадьба


#### Вывод

Большую долю задолженности имеют люди, целью которых является автомобиль - 9.3395%. Почти такой процент у людей с целью образования - 9.1994%. Меньшая склонность к задолженности у людей с целью свадьбы и недвижимости, в одном случае 7.9216%, в другом - 7.214%.

## Общий вывод

Приведя данные в порядок, их анализ показал, что к задолженности по кредиту склоны больше всего люди со средним доходом, имеющие детей, не женатые / не замужем, целью которых является автомобиль или образование.

Вопросы и гипотезы для проверки:
- наличие детей увеличивает вероятность получения задолженности по кредиту - 9.2%;
- полные семьи больше всего склоны к задолженности по кредиту - 8-9%;
- не женатые / не замужем люди больше всего склоны к задолженности по кредиту - 9.7%;
- средний класс больше всего склонен как и класс с низким доходом к задолженности по кредиту - 8%;
- больше всего склоны к задолженности по кредиту люди, целью которых является оплата образования и покупка автомобиля - 9%.

Банку стоит иметь в виду полученные вывод при принятии решений о выдаче кредита людям, входящим в эти категории. Как вариант, снизить количество выдаваемых кредитов людям из данных категорий.